In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
train_data.info()
train_data.head()

In [ ]:
test_data.info()
test_data.head()

In [ ]:
pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
X = train_data.drop(['id', 'target'], axis=1)
y = train_data['target']

# Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(10, random_state=0, shuffle=True)

space = {'penalty': ['l1', 'l2', 'elasticnet'],
         'C': [0.0001, 0.001, 0.01, 0.1, 1, 10]}

model = LogisticRegression(random_state=0)

result = RandomizedSearchCV(model, space, cv=cv, n_iter=10, scoring='neg_log_loss', n_jobs=-1)
result.fit(X_train, y_train)

In [ ]:
print(result.best_score_)
print(result.best_params_)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(penalty=result.best_params_['penalty'], C=result.best_params_['C'], random_state=0)

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import log_loss

log_loss(y_train, logreg.predict_proba(X_train))

In [ ]:
log_loss(y_test, logreg.predict_proba(X_test))

In [ ]:
train_data.info()
train_data.head()

In [ ]:
logreg.predict_proba(train_data.drop(['id', 'target'], axis=1))

In [ ]:
predictions = pd.merge(test_data['id'], pd.DataFrame(logreg.predict_proba(test_data.drop(['id'], axis=1)), columns=['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']), left_index=True, right_index=True)

In [ ]:
predictions.info()
predictions.head()

In [ ]:
predictions.to_csv('submission.csv', index=False)